In [1]:
# ==========================================================
# BLOCK 0️ – IMPORT MODULES
# ==========================================================

import os
import re
import datetime
import time
import pandas as pd
import numpy as np

In [2]:
# ==========================================================
# BLOCK 1️ – DEFINE PROJECT FOLDER STRUCTURE
# ==========================================================

base_path = r"C:\Users\shamil.iqbal\Downloads\Todays Reports"

sales_folder = os.path.join(base_path, "CSV")
reference_folder = os.path.join(base_path, "Reference")
processed_folder = os.path.join(base_path, "Processed")

header_path = os.path.join(reference_folder, "Header.csv")
outl_path = os.path.join(reference_folder, "OUTL.csv")

print("Paths defined successfully.")

Paths defined successfully.


In [3]:
# ==========================================================
# BLOCK 2️⃣ – LOAD HEADER FILE
# ==========================================================

header_cols = pd.read_csv(
    header_path,
    sep="\t",
    header=None,
    encoding_errors="ignore"
).iloc[0].tolist()

header_cols = [str(col).strip() for col in header_cols]

print("Header column count:", len(header_cols))

Header column count: 87


In [4]:
# ==========================================================
# BLOCK 3️⃣ – DEFINE REQUIRED SALES COLUMNS
# ==========================================================

desired_cols = [
    'Company', 'CompName',
    'Store','StoreName','SKU','SkuDesc','SkuStatus',
    'Supplier','SupplierName','SupplierType',
    'Dept.','DeptName','SubDept.','SubDeptName',
    'Class','ClassName','SubClass','SubClassName',
    'Brand','BrandName',
    'NetVndrUnitCost','SOHQty','InvValue',
    'POonOrdrQty','TransferonOrderQty',
    '30DaysQtySold','60DaysQtySold','90DaysQtySold'
]

missing = [c for c in desired_cols if c not in header_cols]
if missing:
    print("⚠ Missing columns:", missing)
else:
    print("All required columns found.")

All required columns found.


In [5]:
# ==========================================================
# BLOCK 4️⃣ – FUNCTION: EXTRACT REPORT DATE FROM FILENAME
# ==========================================================

def extract_date_from_filename(filename):
    match = re.search(r'(\d{8})', filename)
    if match:
        date_str = match.group(1)
        return datetime.datetime.strptime(date_str, "%Y%m%d").date()
    return None

print("Date extraction function ready.")

Date extraction function ready.


In [6]:
# ==========================================================
# BLOCK 5️⃣ – IDENTIFY SALES FILES
# ==========================================================

data_files = [
    f for f in os.listdir(sales_folder)
    if f.endswith(".csv")
]

print("Sales files found:", data_files)

Sales files found: ['SKUDTLBPF_20260226-103743.csv', 'SKUDTLIPF_20260226-103743.csv', 'SKUDTLPF_20260226-103743.csv']


In [7]:
# ==========================================================
# BLOCK 6️ – LOAD SALES FILES (HEAVY PROCESS)
# ==========================================================

chunksize = 300_000
all_chunks = []
grand_total = 0

for i, file in enumerate(data_files, start=1):

    file_path = os.path.join(sales_folder, file)
    report_date = extract_date_from_filename(file)

    print(f"\n🔄 Processing {i}/{len(data_files)}: {file}")
    print(f"   Report Date: {report_date}")

    start_time = time.time()
    file_total = 0

    for chunk in pd.read_csv(
        file_path,
        sep="\t",
        header=None,
        names=header_cols,
        usecols=desired_cols,
        chunksize=chunksize,
        encoding_errors="ignore",
        low_memory=False
    ):
        file_total += len(chunk)
        grand_total += len(chunk)

        chunk["ReportCreationDate"] = report_date
        chunk["Source_File"] = file

        all_chunks.append(chunk)

        print(f"   Loaded {file_total:,} rows (file) | {grand_total:,} rows (total)")

    print(f"   ✅ Finished {file} in {round(time.time() - start_time, 2)} seconds")


# ==========================================================
# BLOCK 7️⃣ A – COMBINE SALES DATA
# ==========================================================

print("\n🔄 Combining all sales files...")
combined_df = pd.concat(all_chunks, ignore_index=True)

print("✅ Final Sales Shape:", combined_df.shape)


🔄 Processing 1/3: SKUDTLBPF_20260226-103743.csv
   Report Date: 2026-02-26
   Loaded 300,000 rows (file) | 300,000 rows (total)
   Loaded 544,216 rows (file) | 544,216 rows (total)
   ✅ Finished SKUDTLBPF_20260226-103743.csv in 4.62 seconds

🔄 Processing 2/3: SKUDTLIPF_20260226-103743.csv
   Report Date: 2026-02-26
   Loaded 300,000 rows (file) | 844,216 rows (total)
   Loaded 600,000 rows (file) | 1,144,216 rows (total)
   Loaded 900,000 rows (file) | 1,444,216 rows (total)
   Loaded 1,200,000 rows (file) | 1,744,216 rows (total)
   Loaded 1,500,000 rows (file) | 2,044,216 rows (total)
   Loaded 1,800,000 rows (file) | 2,344,216 rows (total)
   Loaded 1,856,754 rows (file) | 2,400,970 rows (total)
   ✅ Finished SKUDTLIPF_20260226-103743.csv in 14.26 seconds

🔄 Processing 3/3: SKUDTLPF_20260226-103743.csv
   Report Date: 2026-02-26
   Loaded 300,000 rows (file) | 2,700,970 rows (total)
   Loaded 600,000 rows (file) | 3,000,970 rows (total)
   Loaded 900,000 rows (file) | 3,300,970 row

In [8]:
# ==========================================================
# BLOCK 7️⃣ B – FILTER DATA (Company 90 + SupplierType L + Dept list)
# ==========================================================

# Make filters robust (handle numbers stored as text / floats)
combined_df["Company"] = combined_df["Company"].astype(str).str.strip()
combined_df["CompName"] = combined_df["CompName"].astype(str).str.strip()
combined_df["SupplierType"] = combined_df["SupplierType"].astype(str).str.strip()

# Dept. sometimes comes as 600 or 600.0, so convert to numeric safely
combined_df["Dept."] = pd.to_numeric(combined_df["Dept."], errors="coerce")

dept_list = [600, 605, 625, 630, 635, 645]

filtered_df = combined_df[
    (combined_df["Company"] == "90") &
    (combined_df["CompName"] == "DANUBE COMPANY - BAHRAIN") &
    (combined_df["SupplierType"] == "L") &
    (combined_df["Dept."].isin(dept_list))
].copy()

print("✅ Filtered Shape:", filtered_df.shape)



✅ Filtered Shape: (17731, 30)


In [9]:
# ==========================================================
# BLOCK 8️⃣ – CREATE DEPT HIERARCHY COLUMN
# ==========================================================

filtered_df["DeptHierarchy"] = (
    filtered_df[["Dept.", "SubDept.", "Class", "SubClass"]]
    .fillna("")
    .astype(str)
    .agg("-".join, axis=1)
)

print("DeptHierarchy column created.")

DeptHierarchy column created.


In [10]:
# ==========================================================
# BLOCK 9️⃣ – DEMAND CALCULATIONS (30/60/90 LOGIC)
# ==========================================================

# Ensure sales columns are numeric
sales_cols = ["30DaysQtySold", "60DaysQtySold", "90DaysQtySold"]

for col in sales_cols:
    filtered_df[col] = pd.to_numeric(
        filtered_df[col],
        errors="coerce"
    ).fillna(0)

# =MAX(AY3,0)
filtered_df["Adj_30Days"] = (
    filtered_df["30DaysQtySold"].clip(lower=0)
)

# =MAX(BB3-AY3,0)
filtered_df["Adj_60Days"] = (
    filtered_df["60DaysQtySold"] -
    filtered_df["30DaysQtySold"]
).clip(lower=0)

# =MAX(BE3-BB3,0)
filtered_df["Adj_90Days"] = (
    filtered_df["90DaysQtySold"] -
    filtered_df["60DaysQtySold"]
).clip(lower=0)

# =AVERAGE(CJ3:CL3)
filtered_df["Avg_30_60_90"] = (
    filtered_df[["Adj_30Days", "Adj_60Days", "Adj_90Days"]]
    .mean(axis=1)
)

# Daily sales (ADF)
filtered_df["Daily_Avg_Sales"] = (
    filtered_df["Avg_30_60_90"] / 30
)

print("Demand metrics calculated successfully.")

total_daily_avg_sales = filtered_df["Daily_Avg_Sales"].sum()

print("Total Daily Avg Sales:", round(total_daily_avg_sales, 2))



Demand metrics calculated successfully.
Total Daily Avg Sales: 1561.8


In [11]:
# ==========================================================
# BLOCK 10  – LOAD OUTL (REFERENCE FILE - Outline Days)
# ==========================================================

outl_df = pd.read_csv(
    outl_path,
    sep=",",
    encoding_errors="ignore"
)

print(f"✅ OUTL file loaded successfully | Rows: {outl_df.shape[0]} | Columns: {outl_df.shape[1]}")


# ==========================================================
# BLOCK 1️⃣1️⃣ – LOAD MASTER & UNIT SIZE REFERENCE FILES
# ==========================================================

# Define file paths
master_path = os.path.join(reference_folder, "Master File Comments.csv")
unitsize_path = os.path.join(reference_folder, "Unit Size.csv")

# Load Master file
master_df = pd.read_csv(
    master_path,
    sep=",",              # adjust if needed
    encoding_errors="ignore"
)

print(f"✅ Master file loaded successfully | Rows: {master_df.shape[0]} | Columns: {master_df.shape[1]}")

# Load Unit Size file
unitsize_df = pd.read_csv(
    unitsize_path,
    sep=",",              # adjust if needed
    encoding_errors="ignore"
)

print(f"✅ Unit Size file loaded successfully | Rows: {unitsize_df.shape[0]} | Columns: {unitsize_df.shape[1]}")


✅ OUTL file loaded successfully | Rows: 462 | Columns: 11
✅ Master file loaded successfully | Rows: 27637 | Columns: 2
✅ Unit Size file loaded successfully | Rows: 23881 | Columns: 3


In [12]:
# ==========================================================
# BLOCK 12 – MAP OUTL OUTL DAYS (Policy Lookup)
# ==========================================================

# Clean spaces
filtered_df["DeptHierarchy"] = filtered_df["DeptHierarchy"].astype(str).str.strip()
outl_df["Con."] = outl_df["Con."].astype(str).str.strip()

# Ensure OUTL is numeric
outl_df["901"] = pd.to_numeric(outl_df["901"], errors="coerce")

# Create lookup (Con. -> 901)
outl_lookup = outl_df.drop_duplicates("Con.").set_index("Con.")["901"]

# Map WITHOUT default first
mapped_days = filtered_df["DeptHierarchy"].map(outl_lookup)

# Identify true missing keys
missing_keys = filtered_df.loc[
    mapped_days.isna(), "DeptHierarchy"
].unique()

print("DeptHierarchy not found in OUTL:")
print(missing_keys)

# Apply default AFTER detection
filtered_df["OUTL_901_Days"] = mapped_days.fillna(21)




DeptHierarchy not found in OUTL:
<StringArray>
['600-999-35-5', '600-80-20-10', '600-999-50-1', '600-80-30-15',
   '600-5-3-16', '600-999-30-5',    '600-6-1-1',   '605-40-5-5',
   '605-50-5-5', '625-999-15-5']
Length: 10, dtype: str


In [13]:
# ==========================================================
# BLOCK 12A – CALCULATE OUTL_QTY (Daily Demand × OUTL Days)
# ==========================================================

filtered_df["OUTL_QTY"] = (
    filtered_df["Daily_Avg_Sales"] *
    filtered_df["OUTL_901_Days"]
)

# Calculate total OUTL quantity
total_outl_qty = filtered_df["OUTL_QTY"].sum()

print("✅ OUTL_QTY calculated successfully.")
print(f"Total OUTL_QTY: {round(total_outl_qty, 2)}")

✅ OUTL_QTY calculated successfully.
Total OUTL_QTY: 42618.46


In [14]:
# ==========================================================
# BLOCK 13 – MAP SET QUANTITY USING PARENT SKU
# ==========================================================

# Clean
filtered_df["SKU"] = filtered_df["SKU"].astype(str).str.strip()
unitsize_df["Parent SKU"] = unitsize_df["Parent SKU"].astype(str).str.strip()

unitsize_df["Set Quantity"] = pd.to_numeric(unitsize_df["Set Quantity"], errors="coerce")

# Lookup
parent_lookup = (
    unitsize_df
    .drop_duplicates("Parent SKU")
    .set_index("Parent SKU")["Set Quantity"]
)

# Map
filtered_df["Set_Qty"] = filtered_df["SKU"].map(parent_lookup)

# Track missing before default
filtered_df["Set_Qty_Missing"] = filtered_df["Set_Qty"].isna()

# Apply default = 1
filtered_df["Set_Qty"] = filtered_df["Set_Qty"].fillna(1)

total_rows = len(filtered_df)
defaulted_rows = filtered_df["Set_Qty_Missing"].sum()

print("✅ Set_Qty mapped successfully.")
print(f"Defaulted to 1: {defaulted_rows} out of {total_rows} rows")

# Calculate total SET quantity
total_Set_Qty_qty = filtered_df["Set_Qty"].sum()

print(f"Total SET_QTY: {round(total_Set_Qty_qty, 2)}")



✅ Set_Qty mapped successfully.
Defaulted to 1: 6473 out of 17731 rows
Total SET_QTY: 270751.8


In [15]:
# ==========================================================
# EXPORT DUPLICATE Master SKUs TO EXCEL
# ==========================================================

dup_skus_df = master_df[
    master_df.duplicated(subset=["SKU"], keep=False)
]

print("Duplicate rows found:", len(dup_skus_df))

dup_output_path = os.path.join(base_path, "Processed", "Duplicate_Master_SKUs.xlsx")

dup_skus_df.to_excel(dup_output_path, index=False)

print(f"✅ Duplicate Master SKU file saved successfully.")

Duplicate rows found: 207
✅ Duplicate Master SKU file saved successfully.


In [16]:
# ==========================================================
# BLOCK 14 – MAP COMMENTS WITH STATUS CONDITION
# ==========================================================

# Clean SKUs
filtered_df["SKU"] = (
    filtered_df["SKU"]
    .astype("string")
    .str.strip()
)

master_df["SKU"] = (
    master_df["SKU"]
    .astype("string")
    .str.strip()
)

# Build lookup: SKU -> Comment
comments_lookup = (
    master_df
    .drop_duplicates("SKU")
    .set_index("SKU")["Comment"]
)

# --- CONDITIONAL MAPPING ---
filtered_df["Comment"] = filtered_df["SKU"].map(comments_lookup)

# If SkuStatus != "A", set "Inactive"
filtered_df.loc[
    filtered_df["SkuStatus"] != "A",
    "Comment"
] = "Inactive"

# Missing tracking (only for active SKUs)
total_rows = len(filtered_df)
missing_comments = filtered_df["Comment"].isna().sum()

print("✅ Comments mapped with status condition.")
print(f"Missing Comments (Active only): {missing_comments} out of {total_rows} rows")

✅ Comments mapped with status condition.
Missing Comments (Active only): 35 out of 17731 rows


In [17]:


# ==========================================================
# BLOCK 15 – CALCULATE NET NEEDED QTY (Excel Logic)
# ==========================================================

# Ensure numeric inputs (avoid errors like IFERROR)
num_cols = ["OUTL_QTY", "SOHQty", "POonOrdrQty", "TransferonOrderQty", "Set_Qty"]
for c in num_cols:
    if c in filtered_df.columns:
        filtered_df[c] = pd.to_numeric(filtered_df[c], errors="coerce").fillna(0)

# If your "UNIT SIZE" column in Excel corresponds to Set_Qty in Python:
# (If you named it differently, change here)
filtered_df["UNIT SIZE"] = filtered_df["Set_Qty"]

# Supply = SUM(MAX(AT,0), MAX(AV,0), MAX(AW,0))
supply = (
    filtered_df["SOHQty"].clip(lower=0) +
    filtered_df["POonOrdrQty"].clip(lower=0) +
    filtered_df["TransferonOrderQty"].clip(lower=0)
)

# Raw need = OUTL_QTY - Supply
raw_need = filtered_df["OUTL_QTY"] - supply

# Need = IFERROR(MAX(ROUNDUP(raw_need,0),0),0)
need_units = np.ceil(raw_need).clip(lower=0)

need_mult_6 = (np.floor(need_units / 6 + 0.5) * 6)

# Final Net Needed QTY (Excel IF condition)
filtered_df["Net Needed QTY"] = np.where(
    filtered_df["UNIT SIZE"] == 1,
    need_mult_6,
    need_units
)

print("✅ Net Needed QTY calculated.")

filtered_df["Value"] = (
    pd.to_numeric(filtered_df["NetVndrUnitCost"], errors="coerce").fillna(0)
    *
    pd.to_numeric(filtered_df["Net Needed QTY"], errors="coerce").fillna(0)
)

print("✅ Value column calculated successfully.")


✅ Net Needed QTY calculated.
✅ Value column calculated successfully.


In [18]:
import os

# ==========================================================
# BLOCK 16 – FINAL OUTPUT VALIDATION + EXPORT
# ==========================================================

# 0) Clean any hidden spaces in column names (very common)
filtered_df.columns = filtered_df.columns.str.strip()

# 1) These are the columns (as-is) that represent your required output data
#    (No renaming; we just pick the correct columns that hold the right meaning)
final_cols = [
    "Store", "StoreName", "SKU", "SkuDesc", "SkuStatus",
    "Supplier", "SupplierName", "SupplierType",
    "Dept.", "DeptName", "SubDeptName", "ClassName", "SubClassName",
    "Brand", "BrandName",
    "SOHQty", "InvValue", "POonOrdrQty",
    "Adj_30Days", "Adj_60Days", "Adj_90Days",          # = 30 Days P1/P2/P3 logic
    "Avg_30_60_90",                                   # = AVG Sold QTY
    "Daily_Avg_Sales",                                # = ADF
    "DeptHierarchy",                                  # = Con.
    "OUTL_901_Days",                                  # = OUTL Days
    "OUTL_QTY",                                       # = OUTL QTY
    "UNIT SIZE",                                      # = UNIT SIZE
    "Net Needed QTY",                                 # = Net Needed QTY
    "Value",                                          # = Value
    "Comment"                                         # = Comments (your column name)
]


final_df = filtered_df[final_cols].copy()

print(f"✅ Final output ready | Rows: {final_df.shape[0]} | Columns: {final_df.shape[1]}")



output_path = os.path.join(base_path, "Processed", "Master_to_Send.csv")

final_df.to_csv(output_path, index=False)

print("✅ File saved as CSV successfully.")

✅ Final output ready | Rows: 17731 | Columns: 30
✅ File saved as CSV successfully.


In [19]:
# ==========================================================
# BLOCK 17 – Excludes DSD, Manual PO, Inactive, Blank, 0 Needed
# ==========================================================



NEED_COL = "Net Needed QTY"

# Ensure Needed Qty is numeric
final_df[NEED_COL] = pd.to_numeric(final_df[NEED_COL], errors="coerce").fillna(0)

# Create clean version of Comment for safe filtering
comment_clean = (
    final_df["Comment"]
    .astype("string")
    .str.strip()
    .str.lower()
)

# Define comments to exclude
Ex_comments = {"dsd", "manual po", "inactive"}

# Keep only valid replenishment rows
final_df = final_df[
    comment_clean.notna()               # remove null comments
    & (comment_clean != "")             # remove blank comments
    & (~comment_clean.isin(Ex_comments))# remove excluded types
    & (final_df[NEED_COL] > 0)          # ✅ remove zero-needed qty
]

print("✅ Data preparation completed — non-replenishment + zero-need rows removed.")
print(f"Remaining rows: {len(final_df):,}")

✅ Data preparation completed — non-replenishment + zero-need rows removed.
Remaining rows: 672


In [20]:
# ==========================================================
# BLOCK 18 – Adding Buyer Code as per comments
# ==========================================================

final_df["Buyer"] = ""

final_df.loc[
    final_df["Comment"].str.startswith("By Segment", na=False),
    "Buyer"
] = "ASCO"

final_df.loc[
    final_df["Comment"].str.strip().str.lower().eq("under buyer id"),
    "Buyer"
] = "A002"

print("✅ Buyer column assigned (exact match rule applied).")

null_or_blank = (
    final_df["Buyer"].isna() |
    (final_df["Buyer"].str.strip() == "")
)

print("Rows where Buyer is NULL or blank:")
print(final_df[null_or_blank].head(10))

✅ Buyer column assigned (exact match rule applied).
Rows where Buyer is NULL or blank:
Empty DataFrame
Columns: [Store, StoreName, SKU, SkuDesc, SkuStatus, Supplier, SupplierName, SupplierType, Dept., DeptName, SubDeptName, ClassName, SubClassName, Brand, BrandName, SOHQty, InvValue, POonOrdrQty, Adj_30Days, Adj_60Days, Adj_90Days, Avg_30_60_90, Daily_Avg_Sales, DeptHierarchy, OUTL_901_Days, OUTL_QTY, UNIT SIZE, Net Needed QTY, Value, Comment, Buyer]
Index: []

[0 rows x 31 columns]


In [21]:
# ==========================================================
# BLOCK 19 – Calculating PO value and Filter-out < 10 SAR
# ==========================================================

final_df["PO_Value"] = (
    final_df
    .groupby(
        ["Buyer", "Comment", "Supplier"],
        dropna=False
    )["Value"]
    .transform("sum")
)

final_df = final_df[final_df["PO_Value"] >= 10]

print("✅ Rows with PO_Value < 10 removed.")
print(f"Remaining rows: {len(final_df)}")

✅ Rows with PO_Value < 10 removed.
Remaining rows: 621


In [22]:
# ==========================================================
# BLOCK 19 – Loading Piece SKU and MOQ Baharain Files
# ==========================================================

min_path = os.path.join(reference_folder, "Bahrain Minimum.xlsx")

minimum_df = pd.read_excel(min_path)

print(f"✅ Bahrain Minimum loaded | Rows: {minimum_df.shape[0]} | Columns: {minimum_df.shape[1]}")

moq_path = os.path.join(reference_folder, "MOQ FOR BAHRAIN.xlsx")

moq_df = pd.read_excel(moq_path)

print(f"✅ MOQ FOR BAHRAIN loaded | Rows: {moq_df.shape[0]} | Columns: {moq_df.shape[1]}")



✅ Bahrain Minimum loaded | Rows: 149 | Columns: 9
✅ MOQ FOR BAHRAIN loaded | Rows: 189 | Columns: 2


In [23]:
# ==========================================================
# BLOCK 19 – Converting Box to Piece SKU where neeeded
# ==========================================================

import pandas as pd

FINAL_SKU_COL = "SKU"
FINAL_NEED_COL = "Net Needed QTY"

MIN_CARTON_COL = "Ctn Code"
MIN_PIECE_COL  = "Pc Cpde"
MIN_QTY_COL    = "Min Qty"

# ----------------------------
# Clean / standardize keys
# ----------------------------
final_df[FINAL_SKU_COL] = final_df[FINAL_SKU_COL].astype("string").str.strip()
minimum_df[MIN_CARTON_COL] = minimum_df[MIN_CARTON_COL].astype("string").str.strip()
minimum_df[MIN_PIECE_COL] = minimum_df[MIN_PIECE_COL].astype("string").str.strip()

# Ensure numeric qty fields
final_df[FINAL_NEED_COL] = pd.to_numeric(final_df[FINAL_NEED_COL], errors="coerce").fillna(0)
minimum_df[MIN_QTY_COL]  = pd.to_numeric(minimum_df[MIN_QTY_COL], errors="coerce")

# ----------------------------
# Build lookup from Bahrain Minimum: Carton_SKU -> (Piece_SKU, Min Qty)
# ----------------------------
min_map = (
    minimum_df
    .dropna(subset=[MIN_CARTON_COL])
    .drop_duplicates(subset=[MIN_CARTON_COL])
    .set_index(MIN_CARTON_COL)[[MIN_PIECE_COL, MIN_QTY_COL]]
)

# ----------------------------
# Map Piece SKU + Min Qty into final_df
# ----------------------------
final_df["Mapped_Piece_SKU"] = final_df[FINAL_SKU_COL].map(min_map[MIN_PIECE_COL])
final_df["Mapped_Min_Qty"]   = final_df[FINAL_SKU_COL].map(min_map[MIN_QTY_COL])

matched = final_df["Mapped_Piece_SKU"].notna() & final_df["Mapped_Min_Qty"].notna()

print(f"✅ Bahrain Minimum matches: {matched.sum():,} out of {len(final_df):,}")

# ----------------------------
# Apply updates ONLY for matched rows
# ----------------------------
# Swap SKU (Carton SKU -> Piece SKU)
final_df.loc[matched, FINAL_SKU_COL] = final_df.loc[matched, "Mapped_Piece_SKU"]

# Update Needed Qty: Needed * Min Qty
final_df.loc[matched, FINAL_NEED_COL] = (
    final_df.loc[matched, FINAL_NEED_COL] * final_df.loc[matched, "Mapped_Min_Qty"]
)

print("✅ SKU swapped (Carton → Piece) and Net Needed QTY multiplied by Min Qty.")


✅ Bahrain Minimum matches: 1 out of 621
✅ SKU swapped (Carton → Piece) and Net Needed QTY multiplied by Min Qty.


In [24]:
# ==========================================================
# BLOCK 20 – Checking and Updating MOQ lines
# ==========================================================

import numpy as np
import pandas as pd

# ---- Column names ----
FINAL_SKU_COL = "SKU"
NEED_COL = "Net Needed QTY"

MOQ_SKU_COL = "Danube Code"     # ✅ MOQ file SKU column
MOQ_COL = "MOQ"          # change if your MOQ qty column name differs

MAX_EXTRA_ALLOWED = 12

# ---- Clean keys ----
final_df[FINAL_SKU_COL] = final_df[FINAL_SKU_COL].astype("string").str.strip()
moq_df[MOQ_SKU_COL]     = moq_df[MOQ_SKU_COL].astype("string").str.strip()

# ---- Ensure numeric ----
final_df[NEED_COL] = pd.to_numeric(final_df[NEED_COL], errors="coerce").fillna(0)
moq_df[MOQ_COL]    = pd.to_numeric(moq_df[MOQ_COL], errors="coerce")

# ---- Build lookup: SKU2 -> MOQ ----
moq_lookup = (
    moq_df
    .dropna(subset=[MOQ_COL])
    .drop_duplicates(subset=[MOQ_SKU_COL])
    .set_index(MOQ_SKU_COL)[MOQ_COL]
)

# ---- Map MOQ into final_df using SKU ----
final_df["Mapped_MOQ"] = final_df[FINAL_SKU_COL].map(moq_lookup)

matched = (
    final_df["Mapped_MOQ"].notna()
    & (final_df["Mapped_MOQ"] > 0)
    & (final_df[NEED_COL] > 0)
)

print(f"✅ MOQ matched rows: {matched.sum():,} out of {len(final_df):,}")

# ---- Apply MOQ rounding rule (round UP to nearest MOQ multiple) ----
need = final_df.loc[matched, NEED_COL]
moq  = final_df.loc[matched, "Mapped_MOQ"]

adjusted = np.ceil(need / moq) * moq
extra = adjusted - need

# ---- If increase > 12 then set to 0 ----
adjusted_final = np.where(extra > MAX_EXTRA_ALLOWED, 0, adjusted)

# ---- OVERWRITE Net Needed QTY (matched rows only) ----
final_df.loc[matched, NEED_COL] = adjusted_final

print("✅ Net Needed QTY overwritten using MOQ rules")



✅ MOQ matched rows: 12 out of 621
✅ Net Needed QTY overwritten using MOQ rules


In [25]:
# ==========================================================
# BLOCK 21 – Data Export
# ==========================================================


# Columns required at the beginning
priority_cols = [
    "Buyer",
    "Comment",
    "Store",
    "Supplier",
    "SKU",
    "Net Needed QTY"
]

# Keep only those that actually exist (safety check)
priority_cols = [c for c in priority_cols if c in final_df.columns]

# Remaining columns (preserve order)
remaining_cols = [c for c in final_df.columns if c not in priority_cols]

# Reorder DataFrame
final_df = final_df[priority_cols + remaining_cols]

print("✅ Columns reordered successfully.")



output_path = os.path.join(base_path, "Processed", "Upload_File.csv")

final_df.to_csv(output_path, index=False)

print("✅ File saved as CSV successfully.")

✅ Columns reordered successfully.
✅ File saved as CSV successfully.
